In [84]:
import struct

def compress_lzw(data, bit_width=12):
    dictionary = {chr(i): i for i in range(256)}
    result = []
    w = ""
    next_code = 256
    max_code = 2**bit_width - 1

    for c in data:
        wc = w + c
        if wc in dictionary:
            w = wc
        else:
            result.append(dictionary[w])
            if next_code <= max_code:
                dictionary[wc] = next_code
                next_code += 1
            w = c

    if w:
        result.append(dictionary[w])

    # Pack the codes into a hex string
    packed = ""
    buffer = 0
    bits_in_buffer = 0
    for code in result:
        buffer |= code << bits_in_buffer
        bits_in_buffer += bit_width
        while bits_in_buffer >= 8:
            packed += f"{buffer & 0xFF:02x}"
            buffer >>= 8
            bits_in_buffer -= 8
    if bits_in_buffer > 0:
        packed += f"{buffer & 0xFF:02x}"

    return packed

def decompress_lzw(compressed_hex, bit_width=12):
    dictionary = {i: chr(i) for i in range(256)}
    next_code = 256
    max_code = 2**bit_width - 1

    # Convert hex string to bytes
    compressed = bytes.fromhex(compressed_hex)

    # Unpack the byte string into codes
    result = []
    buffer = 0
    bits_in_buffer = 0
    code = 0
    for byte in compressed:
        buffer |= byte << bits_in_buffer
        bits_in_buffer += 8
        while bits_in_buffer >= bit_width:
            code = buffer & ((1 << bit_width) - 1)
            buffer >>= bit_width
            bits_in_buffer -= bit_width
            result.append(code)

    output = ""
    w = chr(result.pop(0))
    output += w

    for k in result:
        if k in dictionary:
            entry = dictionary[k]
        elif k == next_code:
            entry = w + w[0]

        output += entry

        if next_code <= max_code:
            dictionary[next_code] = w + entry[0]
            next_code += 1

        w = entry

    return output

def find_optimal_bit_width(original, min_width=9, max_width=16):
    best_ratio = float('inf')
    best_width = min_width
    results = []

    for bit_width in range(min_width, max_width + 1):
        compressed = compress_lzw(original, bit_width=bit_width)
        decompressed = decompress_lzw(compressed, bit_width=bit_width)
        ratio = len(compressed) / len(original)
        
        results.append({
            'bit_width': bit_width,
            'compressed_length': len(compressed),
            'ratio': ratio,
            'matches_original': original == decompressed
        })
        
        if ratio < best_ratio and original == decompressed:
            best_ratio = ratio
            best_width = bit_width

    return best_width, results


# Example usage
original = """
...
""".replace('\n', '')

# Example usage
def print_compressed(compressed):
    for i in range(0, len(compressed), 256):
        print(compressed[i:i+256])

def find_optimal_bit_width(original, min_width=1, max_width=36):
    best_ratio = float('inf')
    best_width = min_width
    best_compressed = ""

    for bit_width in range(min_width, max_width + 1):
        compressed = compress_lzw(original, bit_width=bit_width)
        decompressed = decompress_lzw(compressed, bit_width=bit_width)
        ratio = len(compressed) / len(original)
        
        if ratio < best_ratio and original == decompressed:
            best_ratio = ratio
            best_width = bit_width
            best_compressed = compressed

    return best_width, best_compressed, best_ratio

# Example usage
best_width, compressed, ratio = find_optimal_bit_width(original, min_width=3, max_width=26)

print(f"Best bit width: {best_width}")
print(f"Original length: {len(original)}")
print(f"Compressed length: {len(compressed)}")
print(f"Compression ratio: {ratio:.2f}")
print("\nCompressed data:")
print_compressed(compressed)

# 16384

Best bit width: 12
Original length: 28672
Compressed length: 16024
Compression ratio: 0.56

Compressed data:
3070030021036130100541100761100981100ba1100d71100001103530033760031451033100103780113750031561031d51033450033030121761111b11112241122b51122451031701121ed1122f31132b410363400362301331701210c1112211033161112c611338a1132c11141f21133cf11128f11323711302c11053e1
1003010330901330a11416f1124441132cd1152d11153760150161163e81160171153010036490163ed1166f711614410336000376111076611735200320310378711781a1171031187c41118031187921188b31188dc1186240033940031431068811188661038ea1198201199241199b61197b21038771179d511669a1116b
b11671a11a30301035f0198c8116a04118a51119a731177201116b31036300036e410605e116641106198111577103a9c111668015366006ce710365e01cd3610637f01c661003622003628015dd5117d7011dd2011ee3411de5f11c64601eced11cd5210676111993a117654003e2311de8411ef6511ef881159e6117ebb11e
e9c11c34f01f2491033ea11cadb11632300633e01608821c343011658015378003d6411dd16103648021d5511df481150f611607921b67a11a231214

In [78]:
compressed

'3070030021036130100541100761100981100ba1100d71100001103530033760031451033100103780113750031561031d51033450033030121761111b11112241122b51122451031701121ed1122f31132b410363400362301331701210c1112211033161112c611338a1132c11141f21133cf11128f11323711302c11053e11003010330901330a11416f1124441132cd1152d11153760150161163e81160171153010036490163ed1166f711614410336000376111076611735200320310378711781a1171031187c41118031187921188b31188dc1186240033940031431068811188661038ea1198201199241199b61197b21038771179d511669a1116bb11671a11a30301035f0198c8116a04118a51119a731177201116b31036300036e410605e116641106198111577103a9c111668015366006ce710365e01cd3610637f01c661003622003628015dd5117d7011dd2011ee3411de5f11c64601eced11cd5210676111993a117654003e2311de8411ef6511ef881159e6117ebb11ee9c11c34f01f2491033ea11cadb11632300633e01608821c343011658015378003d6411dd16103648021d5511df481150f611607921b67a11a2312143160030c8216c8811a654021371006307203392023154223311206d9110395711dd6312332822340522342122461902144a21e02a2247bc